# Testing the Left Arm

**Warning:** Before running this notebook, first make sure you understand the command you run and make sure that the robot can freely move.

**Note:** Also stop all other running Python script or notebook connected to the robot as only one connection can run at the same time.

In [1]:
%matplotlib notebook

import time
import numpy as np0

from matplotlib import pyplot as plt

## Connect to the left arm

In [2]:
from reachy import Reachy, parts

In [4]:
reachy = Reachy(
    left_arm=parts.LeftArm(io='/dev/ttyUSB*', hand='force_gripper'),
    #left_arm=parts.LeftArm(io='ws', hand='force_gripper'),
)

LuosGateNotFoundError: Gate "r_left_arm" not found on ports "/dev/ttyUSB*"

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/venvs/reachy-env/lib/python3.7/site-packages/pyluos/io/serial_io.py", line 95, in _poll
    s = self._serial.read(to_read)
  File "/home/pi/venvs/reachy-env/lib/python3.7/site-packages/serial/serialposix.py", line 596, in read
    'device reports readiness to read but returned no data '
serial.serialutil.SerialException: device reports readiness to read but returned no data (device disconnected or multiple access on port?)

Exception in thread Thread-6:
Traceback (most recent call last):
  File "/usr/lib/python3.7/threading.py", line 917, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.7/threading.py", line 865, in run
    self._target(*self._args, **self._kwargs)
  File "/home/pi/venvs/reachy-env/lib/pyt

You can now connect your robot in Unity.

## Check motors

Check that all motors are presents

In [ ]:
for m in reachy.left_arm.motors:
    print(f'{m.name} - pos:{m.present_position} temp:{m.temperature}')

Move the arm and check that the position are moving

## Turn stiff/compliant

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = False

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = True

Check that all motors are turning compliant/stiff.

## Goto 0 pos + 90 on the elbow

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = False

In [ ]:
reachy.goto({
    'left_arm.shoulder_pitch': 0,
    'left_arm.shoulder_roll': 0,
    'left_arm.arm_yaw': 0,    
    'left_arm.elbow_pitch': -90,
    'left_arm.hand.forearm_yaw': 0,
    'left_arm.hand.wrist_pitch': 0,
    'left_arm.hand.wrist_roll': 0,
    'left_arm.hand.gripper': 0,
}, duration=3, wait=True)

Check that everything seems fine.

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = True

## Record a trajectory and replay it

*This part doesn't really seem to work with the simulator. - PC*

In [ ]:
from reachy.trajectory import TrajectoryRecorder, TrajectoryPlayer

In [ ]:
recorder = TrajectoryRecorder(reachy.left_arm.motors)

We will record a move for 10sec. Make sure you hold the robot away from all objects before starting the record.

In [ ]:
recorder.start()
time.sleep(10)
recorder.stop()

Turn all motors stiff (try putting the robot in about its starting position of the trajectory).

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = False

In [ ]:
player = TrajectoryPlayer(reachy, recorder.trajectories)
player.play(wait=True, fade_in_duration=3)

Turn it back compliant.

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = True

Look at the recorded traj.

In [ ]:
plt.figure()

for k, v in recorder.trajectories.items():
    plt.plot(v, label=k)
# plt.legend()

## Grasp objects

Put an object inside the hand of the robot and then run the line below to close it.

In [ ]:
reachy.left_arm.hand.gripper.compliant = False

In [ ]:
reachy.left_arm.hand.close()

Check that the close method returned true. Check the object holds in its hand while moving the arm manually.

*This returns false because you can't really put in anything in Reachy's hand in the simulator. - PC*

Then we reopen the hand.

In [ ]:
reachy.left_arm.hand.open()

In [ ]:
reachy.left_arm.hand.gripper.compliant = True

## Check kinematics

Put the arm in a position where its hand is facing forward and where it can move at least 10cm in all direction.

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = False

In [ ]:
J0 = [m.present_position for m in reachy.left_arm.motors]

M = reachy.left_arm.forward_kinematics(J0)

M1 = M.copy()
M1[0, 3] += 0.05
J1 = reachy.left_arm.inverse_kinematics(M1, J0)

M2 = M.copy()
M2[0, 3] -= 0.05
J2 = reachy.left_arm.inverse_kinematics(M2, J0)

M3 = M.copy()
M3[1, 3] += 0.05
J3 = reachy.left_arm.inverse_kinematics(M3, J0)

M4 = M.copy()
M4[1, 3] -= 0.05
J4 = reachy.left_arm.inverse_kinematics(M4, J0)

In [ ]:
_ = reachy.goto({m.name: j for j, m in zip(J1, reachy.left_arm.motors)}, duration=1, wait=True)
_ = reachy.goto({m.name: j for j, m in zip(J2, reachy.left_arm.motors)}, duration=1, wait=True)
_ = reachy.goto({m.name: j for j, m in zip(J3, reachy.left_arm.motors)}, duration=1, wait=True)
_ = reachy.goto({m.name: j for j, m in zip(J4, reachy.left_arm.motors)}, duration=1, wait=True)
_ = reachy.goto({m.name: j for j, m in zip(J0, reachy.left_arm.motors)}, duration=1, wait=True)

In [ ]:
for m in reachy.left_arm.motors:
    m.compliant = True

## Check the fans

*I'm not really sure if this does anything in the simulator. - PC*

In [ ]:
reachy.left_arm.shoulder_fan.on()

In [ ]:
reachy.left_arm.shoulder_fan.off()

In [ ]:
reachy.left_arm.elbow_fan.on()

In [ ]:
reachy.left_arm.elbow_fan.off()

In [ ]:
reachy.left_arm.wrist_fan.on()

In [ ]:
reachy.left_arm.wrist_fan.off()